In [3]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species
from rmgpy import settings

import os.path
import logging
import codecs
from copy import deepcopy
from IPython.display import display

from rmgpy.constraints import failsSpeciesConstraints
from rmgpy.data.base import Database, Entry, LogicNode, LogicOr, ForbiddenStructures,\
                            getAllCombinations
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius
from rmgpy.molecule import Bond, GroupBond, Group, Molecule
from rmgpy.species import Species
import rmgpy.species as species

from rmgpy.data.kinetics.common import saveEntry
from rmgpy.data.kinetics.depository import KineticsDepository
from rmgpy.data.kinetics.groups import KineticsGroups
from rmgpy.data.kinetics.rules import KineticsRules
import rmgpy.data.thermo as thermo
from rmgpy.exceptions import InvalidActionError, ReactionPairsError, KineticsError,\
                             UndeterminableKineticsError, ForbiddenStructureException,\
                             KekulizationError, ActionError

In [21]:
adj="""1 C u0 p0 c0 {2,D} {3,S} {4,S}
2 O u0 p2 c0 {1,D}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}"""

spc = Species().fromAdjacencyList(adj)
display(spc)

newMol = spc.getResonanceHybrid()
display(newMol)
for atom in newMol.vertices:
    if atom.isNonHydrogen():
        print atom
        for atom2, bond in atom.edges.items():
            print bond.order

C
1.5
1.0
1.0
O
1.5


In [2]:
adj="""1 C u0 p0 c0 {2,D} {3,S} {4,S}
2 O u0 p2 c0 {1,D}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}"""

spc = Species().fromAdjacencyList(adj)
display(spc)

newMol = spc.getResonanceHybrid()
display(newMol)
for atom in newMol.vertices:
    if atom.isNonHydrogen():
        print atom
        for atom2, bond in atom.edges.items():
            print bond.order

C
1.0
1.0
2.0
O
2.0


In [23]:
len([[Species(label="[O]", molecule=[Molecule(SMILES="[O]")])], [Species(label="O=S=O", molecule=[Molecule(SMILES="O=S=O")])]])


2

In [12]:
smiles = 'O[N]'
adj="""multiplicity 3
1 O u0 p2 c0 {2,S} {3,S}
2 H u0 p0 c0 {1,S}
3 N u2 p1 c0 {1,S}"""
# mol = Molecule(SMILES=smiles)
mol = Molecule().fromAdjacencyList(adj)

In [20]:
molList = [mol]

index = 0
while index < len(molList):
    molecule = molList[index]
    newMolList = []
    m = molList[index]
    
    print "exploring:"
    display(m)
    
    isomers = []
    
    # findAllDelocalizationPaths
    paths = []
    for atom1 in m.vertices:
        for atom2, bond12 in atom1.edges.items():
            # Vinyl bond must be capable of gaining an order
            if (bond12.isSingle() or bond12.isDouble()) and (atom1.radicalElectrons in [1, 2]):
                for atom3, bond23 in atom2.edges.items():
                    # Allyl bond must be capable of losing an order without breaking
                    if atom1 is not atom3 and (bond23.isDouble() or bond23.isTriple()):
                        paths.append([atom1, atom2, atom3, bond12, bond23])
    
    for atom1, atom2, atom3, bond12, bond23 in paths:
        # Adjust to (potentially) new resonance isomer
        atom1.decrementRadical()
        atom3.incrementRadical()
        bond12.incrementOrder()
        bond23.decrementOrder()
        # Make a copy of isomer
        isomer = m.copy(deep=True)
        # Also copy the connectivity values, since they are the same
        # for all resonance forms
        for index in range(len(mol.vertices)):
            v1 = m.vertices[index]
            v2 = isomer.vertices[index]
            v2.connectivity1 = v1.connectivity1
            v2.connectivity2 = v1.connectivity2
            v2.connectivity3 = v1.connectivity3
            v2.sortingLabel = v1.sortingLabel
        # Restore current isomer
        atom1.incrementRadical()
        atom3.decrementRadical()
        bond12.decrementOrder()
        bond23.incrementOrder()
#         print "allyl isomer:"
#         display(isomer)
        try:
            isomer.updateAtomTypes(logSpecies=False)
        except:
            pass  # Don't append resonance structure if it creates an undefined atomType
        else:
            for atom in isomer.atoms:
                print atom.atomType.label
            isomers.append(isomer)
    
    
    # findAllDelocalizationPathsLonePairRadical
    paths = []
    for atom1 in m.vertices:
        if (atom1.isNitrogen() and atom1.radicalElectrons >= 1 and atom1.lonePairs in [0, 1]
            or atom1.isSulfur() and atom1.radicalElectrons >= 1 and atom1.lonePairs in [0, 1, 2]):
            for atom2, bond12 in atom1.edges.items():
                if bond12.isSingle():
                    if ((atom2.isNitrogen() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2])
                        or (atom2.isOxygen() and atom2.radicalElectrons == 0 and atom2.lonePairs == 3)
                        or (atom2.isSulfur() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2, 3])):
                        paths.append([atom1, atom2])
        # O is treated separately to avoid RO[::O.] <-> R[:O.+][:::O-]
        elif atom1.isOxygen() and atom1.radicalElectrons >= 1 and atom1.lonePairs == 2:
            for atom2, bond12 in atom1.edges.items():
                if bond12.isSingle():
                    if ((atom2.isNitrogen() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2])
                        or (atom2.isSulfur() and atom2.radicalElectrons == 0 and atom2.lonePairs in [1, 2, 3])):
                        paths.append([atom1, atom2])
        
    for atom1, atom2 in paths:
        # Adjust to (potentially) new resonance isomer
        atom1.decrementRadical()
        atom1.incrementLonePairs()
        atom1.updateCharge()
        atom2.incrementRadical()
        atom2.decrementLonePairs()
        atom2.updateCharge()
        # Make a copy of isomer
        isomer = m.copy(deep=True)
        # Also copy the connectivity values, since they are the same
        # for all resonance forms
        for index in range(len(mol.vertices)):
            v1 = m.vertices[index]
            v2 = isomer.vertices[index]
            v2.connectivity1 = v1.connectivity1
            v2.connectivity2 = v1.connectivity2
            v2.connectivity3 = v1.connectivity3
            v2.sortingLabel = v1.sortingLabel
        # Restore current isomer
        atom1.incrementRadical()
        atom1.decrementLonePairs()
        atom1.updateCharge()
        atom2.decrementRadical()
        atom2.incrementLonePairs()
        atom2.updateCharge()
#         print "LPrad isomer:"
#         display(isomer)
        try:
            isomer.updateAtomTypes(logSpecies=False)
        except:
            pass  # Don't append resonance structure if it creates an undefined atomType
        else:
            for atom in isomer.atoms:
                print atom.atomType.label
            isomers.append(isomer)
        
        
        
    # findAllDelocalizationPathsLonePairMultipleBond
    paths = []
    for atom1 in m.vertices:
        for atom2, bond12 in atom1.edges.items():
            if (not (atom1.radicalElectrons and atom2.radicalElectrons)  # not allowing both atom1, atom2 to be radicals
                and atom2.isNonHydrogen()):  # don't bother with hydrogen atoms
                if bond12.isSingle() or bond12.isDouble():
                    # Find paths in the direction <increasing> the bond order
                    # atom1 must posses at least one lone pair to loose it
                    if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [1, 2, 3])
                        or (atom1.isOxygen() and atom1.lonePairs in [2, 3])  # not allowing O p0
                        and (atom2.radicalElectrons or atom2.charge > 0)):  # to increase the bond order atom2 must
                        # either be a radical or have a positive charge
                        paths.append([atom1, atom2, bond12, 1])  # direction = 1
                    for atom3, bond23 in atom2.edges.items():  # check whether this is the two subsequent transformation case
                        if (not (atom1.radicalElectrons or atom2.radicalElectrons or atom3.radicalElectrons)
                                and (((atom3.isNitrogen() or atom3.isSulfur()) and atom3.lonePairs in [0, 1, 2])
                                or (atom3.isOxygen() and atom3.lonePairs in [1, 2]))
                                and not atom1 is atom3):  # not allowing O p0
                            if atom2.atomType.label in ['N5ddc','N5tc']:
                                paths.append([atom1, atom2, atom3, bond12, bond23, 3])  # direction = 3
                if bond12.isDouble() or bond12.isTriple():
                    # Find paths in the direction <decreasing> the bond order
                    # atom1 gains a lone pair, hence cannot have more than two lone pairs
                    if (((atom1.isNitrogen() or atom1.isSulfur()) and atom1.lonePairs in [0, 1, 2])
                        or (atom1.isOxygen() and atom1.lonePairs in [1, 2])):  # not allowing O p0
                        paths.append([atom1, atom2, bond12, 2])  # direction = 2
                  
    for atom1, atom2, bond12, direction in paths:
        if atom1.radicalElectrons:  # the radical could be on either atom
            atom1.decrementRadical()
            atom2.incrementRadical()
        elif atom2.radicalElectrons:
            atom2.decrementRadical()
            atom1.incrementRadical()
        if direction == 1:  # The direction <increasing> the bond order
            atom1.decrementLonePairs()
            bond12.incrementOrder()
        elif direction == 2:  # The direction <decreasing> the bond order
            atom1.incrementLonePairs()
            bond12.decrementOrder()
        elif direction == 3:  # The two subsequent transformation case
            atom1.decrementLonePairs()
            bond12.incrementOrder()
            atom3.incrementLonePairs()
            bond23.decrementOrder()
            atom3.updateCharge()
        atom1.updateCharge()
        atom2.updateCharge()
        # Make a copy of isomer
        isomer = m.copy(deep=True)
        # Also copy the connectivity values, since they are the same
        # for all resonance forms
        for index in range(len(mol.vertices)):
            v1 = m.vertices[index]
            v2 = isomer.vertices[index]
            v2.connectivity1 = v1.connectivity1
            v2.connectivity2 = v1.connectivity2
            v2.connectivity3 = v1.connectivity3
            v2.sortingLabel = v1.sortingLabel
        # Restore current isomer
        if atom1.radicalElectrons:  # the radical could be on either atom
            atom1.decrementRadical()
            atom2.incrementRadical()
        elif atom2.radicalElectrons:
            atom2.decrementRadical()
            atom1.incrementRadical()
        if direction == 1:  # The direction <increasing> the bond order
            atom1.incrementLonePairs()
            bond12.decrementOrder()
        elif direction == 2:  # The direction <decreasing> the bond order
            atom1.decrementLonePairs()
            bond12.incrementOrder()
        elif direction == 3:  # The two subsequent transformation case, N5ddc <=> N5tc
            atom1.incrementLonePairs()
            bond12.decrementOrder()
            atom3.decrementLonePairs()
            bond23.incrementOrder()
            atom3.updateCharge()
        atom1.updateCharge()
        atom2.updateCharge()
#         print "LPMB isomer:"
#         display(isomer)
        try:
            isomer.updateAtomTypes(logSpecies=False)
        except:
            pass  # Don't append resonance structure if it creates an undefined atomType
        else:
            for atom in isomer.atoms:
                print atom.atomType.label
            isomers.append(isomer)

            
            
            
    newMolList = isomers

    for newMol in newMolList:
        # Append to isomer list if unique
        for mol in molList:
            if mol.isIsomorphic(newMol):
                print "isomofphic:"
                display(newMol)
                break
        else:
            molList.append(newMol)
            print "appending newMol:"
            display(newMol)
            print "len(newMol) = {0}".format(len(newMol))

    # Move to next resonance isomer
    index += 1

print "\n\n\nmolList:"
for molecule1 in molList:
    display(molecule1)



exploring:





molList:


In [ ]:
[[<Atom 'S'>, <Atom 'C'>, <Bond "1.0">, 1],
 [<Atom 'S'>, <Atom 'C.'>, <Bond "1.0">, 1],
 [<Atom 'S.'>, <Atom 'C'>, <Bond "1.0">, 1],
 [<Atom 'O'>, <Atom 'C'>, <Bond "2.0">, 2]]


[[<Atom 'S'>, <Atom 'C'>, <Bond "1.0">, 1], 
 [<Atom 'S'>, <Atom 'C.'>, <Bond "1.0">, 1], 
 [<Atom 'S.'>, <Atom 'C'>, <Bond "1.0">, 1], 
 [<Atom 'O'>, <Atom 'C'>, <Bond "2.0">, 2]]

In [ ]:
for isomer in isomers:
    print mol1.isIsomorphic(isomer)

In [ ]:
display(mol)
print mol.toAdjacencyList()
display(mol1)
print mol1.toAdjacencyList()

In [ ]:
database = RMGDatabase()
database.load(
            path = '/home/alongd/Code/RMG-database/input/',
            thermoLibraries = '',
            transportLibraries = '',
            reactionLibraries = '',
            seedMechanisms = '',
            kineticsFamilies = ['CO_Disproportionation'],
            kineticsDepositories = ['training'],
            depository = False, # Don't bother loading the depository information, as we don't use it
)

In [ ]:
for self in database.kinetics.families.values():
    depository = self.getTrainingDepository()
index = max([e.index for e in self.rules.getEntries()] or [0]) + 1

entries = depository.entries.values()
entries.sort(key=lambda x: x.index)
reverse_entries = []

for entry in entries:
    reaction = entry.item
    
    if True:
    
 #   def getReactionTemplate(self, reaction):
        """
        For a given `reaction` with properly-labeled :class:`Molecule` objects
        as the reactants, determine the most specific nodes in the tree that
        describe the reaction.
        """
    
        # Get forward reaction template and remove any duplicates
        forwardTemplate = self.top[:]

        temporary = []
        symmetricTree = False
        for entry in forwardTemplate:
            if entry not in temporary:
                temporary.append(entry)
            else:
                # duplicate node found at top of tree
                # eg. R_recombination: ['Y_rad', 'Y_rad']
                assert len(forwardTemplate)==2 , 'Can currently only do symmetric trees with nothing else in them'
                symmetricTree = True
        forwardTemplate = temporary

        # Descend reactant trees as far as possible
        template = []
        for entry in forwardTemplate:
            # entry is a top-level node that should be matched
            group = entry.item

            # Identify the atom labels in a group if it is not a logical node
            atomList = []
            if not isinstance(entry.item, LogicNode):
                atomList = group.getLabeledAtoms()

            for reactant in reaction.reactants:
                if isinstance(reactant, Species):
                    reactant = reactant.molecule[0]
                # Match labeled atoms
                # Check that this reactant has each of the atom labels in this group. If it is a LogicNode,
                # the atomList is empty and it will proceed directly to the descendTree step.
                if not all([reactant.containsLabeledAtom(label) for label in atomList]):
                    continue # don't try to match this structure - the atoms aren't there!
                # Match structures
                atoms = reactant.getLabeledAtoms()
                # Descend the tree, making sure to match atomlabels exactly using strict = True
                matched_node = self.descendTree(reactant, atoms, root=entry, strict=True)
                if matched_node is not None:
                    template.append(matched_node)
                #else:
                #    logging.warning("Couldn't find match for {0} in {1}".format(entry,atomList))
                #    logging.warning(reactant.toAdjacencyList())

        # Get fresh templates (with duplicate nodes back in)
        forwardTemplate = self.top[:]
        if self.label.lower().startswith('r_recombination'):
            forwardTemplate.append(forwardTemplate[0])

        # Check that we were able to match the template.
        # template is a list of the actual matched nodes
        # forwardTemplate is a list of the top level nodes that should be matched
        if len(template) != len(forwardTemplate):
#            print 'len(template):', len(template)
#            print 'len(forwardTemplate):', len(forwardTemplate)
            msg = 'Unable to find matching template for reaction {0} in reaction family {1}.'.format(str(reaction), str(self)) 
            msg += 'Trying to match {0} but matched {1}'.format(str(forwardTemplate),str(template))
#            print 'reactants'
#            for reactant in reaction.reactants:
#                print reactant.toAdjacencyList() + '\n'
#            print 'products'
#            for product in reaction.products:
#                print product.toAdjacencyList() + '\n'
            raise UndeterminableKineticsError(reaction, message=msg)

        print "template: ", template
    
    

In [ ]:
self

In [ ]:
import rmgpy.molecule as molecule

adj="""multiplicity 2
1    O u0 p2 c0 {2,D}
2    N u0 p1 c0 {1,D} {3,S}
3 *1 O u1 p2 c0 {2,S}"""

reactant = molecule.Molecule().fromAdjacencyList(adj)

atoms = reactant.getLabeledAtoms()

atoms